## Dependencies
***

In [1]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser


***

## Web Scraping 
***

In [41]:
#Settings for accessing Website

executable_path = {"executable_path": "chromedriver.exe"}

browser = Browser ('chrome', **executable_path, headless=False)



### Defining Functions for Data Scraping and Cleaning
***

#### NBA
***

In [86]:
#Pre-defining Variables for NBA scrape

#Empty DataFrame for NBA
NBA_game_df = pd.DataFrame()


#Reference to names
NBA_refer = "basketball-reference"

#URL
NBA_url = "https://www.basketball-reference.com"

#Create Empty List for years we want to scrape for
years = ["1990", "1991", "1992", "1993", "1994", "1995",\
         "1996", "1997", "1998", "1999", "2000","2001", \
         "2002", "2003", "2004", "2005", "2006", "2007",\
         "2008", "2009", "2010", "2011", "2012", "2013",\
         "2014", "2015", "2016"]



In [93]:
#Define our function for scraping NBA Data---> ** need to come back to this and finish it off
def scrape_nba_data(page_url):
    #URL's for both NBA and NCAA
    #Reference global variable: NBA_game_df
    global NBA_game_df
    
    for year in years:

        #Set the rest of the url
        url = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"

        #Visit the NBA Web page
        browser.visit(url)

        #Retrieve the html for the web page
        html = browser.html

        #Use pandas to read html
        year_html = pd.read_html(html, header = 0)

        #Get the second Table with all of the Data
        year_html = year_html[1]

        #Convert into DataFrame
        year_df = pd.DataFrame(year_html)

        #Delete Column of Ranking: "Rk"
        year_df = year_df.rename(columns=({"Rk" : "Year"}))

        #Apply the year to the Year column for each row
        year_df["Year"] = year_df["Year"].apply(lambda x: year)



        #Append to main DataFrame: NBA_game_df
        if NBA_game_df.empty: 
            NBA_game_df = year_df
        else: 
            NBA_game_df = NBA_game_df.append(year_df, ignore_index = True)

        #hoooldd on wait a second, let me put some sleep in it
        time.sleep(1)



In [12]:
#Function for cleaning and merging NBA data

def clean_nba_data (page_html):
    # Calculate PER
    
    # Set Unique ID to players based upon their name



#### NCAA
***

In [80]:
#Predefining variables for NCAA scrape

NCAA_df = pd.DataFrame()
NCAA_refer = "sports-reference"
NCAA_url = "https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min=1993&year_max=2016&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&games_type=A&c1stat=fga&c1comp=gt&order_by=year_id&order_by_asc=Y"

In [81]:
#Function for scraping NCAA data

def scrape_ncaa_data(page_url):
    global NCAA_df
    url = page_url
    browser.visit(url)
    click_count = 0
    
    while NCAA_refer in url:
        try:
            if click_count == 0:

                #Get html of web page
                html = browser.html

                #Read html using pandas
                read_html = pd.read_html(html)

                #Create DF and append to ncaa_df
                page_df = pd.DataFrame(read_html[1])
                NCAA_df = page_df

                #Add to click counter
                click_count += 1
                
                #Sleep
                time.sleep(1)
                
                #click to go to next page
                next_page = browser.find_link_by_text("Next page")
                next_page.click()
                


            else:
                #Get html of web page
                html = browser.html

                #Read html using pandas
                read_html = pd.read_html(html)

                #print(read_html[1])
                page_df = pd.DataFrame(read_html[1])
                NCAA_df = NCAA_df.append(page_df, ignore_index= True)

                #hoooldd on wait a second, let me put some sleep in it
                time.sleep(1)
            
                #click to go to next page
                next_page = browser.find_link_by_text("Next page")
                next_page.click()
        except AttributeError as error:
            print("End of scrape")
            window.close()
            

In [83]:
#Function for cleaning and merging NCAA data ** Will work on this later

def clean_ncaa_data(ncaa_csv):
    #Calculate PER

SyntaxError: unexpected EOF while parsing (<ipython-input-83-87bced7b6067>, line 4)

### Scraping for Data
***

#### NCAA
***




In [89]:
#Scrapety scrape scrape
scrape_ncaa_data(NCAA_url)

KeyboardInterrupt: 

In [65]:
#Test print
NCAA_df
NCAA_copy = NCAA_df

#Confirm last value on Web Page is equal to that of DF
NCAA_df.tail()


,Unnamed: 0_level_0,Totals,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0,Unnamed: 19_level_0,Unnamed: 20_level_0,Unnamed: 21_level_0,Unnamed: 22_level_0,Unnamed: 23_level_0,Unnamed: 24_level_0,Unnamed: 25_level_0
,Rk,Player,Class,Season,Pos,School,Conf,G,MP,FG,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
112161,103854,Stephen Zimmerman Jr.,FR,2015-16,F,Nevada-Las Vegas,MWC,26,680,102,...,101,56,171,227,22,14,51,54,79,272
112162,103855,Vinny Zollo,SR,2015-16,F,Cleveland State,Horizon,29,642,69,...,48,52,112,164,32,14,12,44,74,194
112163,103856,Nisre Zouzoua,FR,2015-16,G,Bryant,NEC,31,939,131,...,89,20,78,98,44,25,8,45,86,397
112164,103857,Jake Zuilhof,SR,2015-16,F,Central Arkansas,Southland,28,710,69,...,76,66,121,187,18,10,37,51,102,177
112165,103858,Johnny Zuppardo,SR,2015-16,F,Mississippi State,SEC,31,358,38,...,6,16,52,68,7,14,5,13,60,98


In [60]:
#Push Raw Data to CSV (So we don't have to scrape again)

NCAA_copy.to_csv("NCAA_raw.csv", index = False)

#### NBA
***

In [94]:
#Scrapety scrape scrape

scrape_nba_data(NBA_url)

In [96]:
#Test print
NBA_game_df

NBA_copy = NBA_game_df

#Confirm last record on Web Page with that on DF
NBA_copy.tail()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
16035,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,...,.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
16036,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,...,.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
16037,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
16038,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,...,.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7
16039,2016,Tyler Zeller,C,26,BOS,60,3,11.8,2.3,4.8,...,.815,1.0,1.9,3.0,0.5,0.2,0.4,0.8,1.6,6.1


In [97]:
#Push Raw Data to csv
NBA_copy.to_csv("NBA_raw.csv", index = False)



### Cleaning Data
***

#### NCAA
***

In [90]:
#Note: Data has not been collected for per game stats, will need to calculate per game stats.

pd.read_csv("NCAA_raw.csv")

,Unnamed: 0_level_0,Totals,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0,Unnamed: 19_level_0,Unnamed: 20_level_0,Unnamed: 21_level_0,Unnamed: 22_level_0,Unnamed: 23_level_0,Unnamed: 24_level_0,Unnamed: 25_level_0
0,Rk,Player,Class,Season,Pos,School,Conf,G,MP,FG,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
2,2,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
3,3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
4,4,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0
5,5,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,NaN,34,...,33,NaN,NaN,29,50,19,3,NaN,NaN,114
6,6,Ahmed Aboul-Foutouh,JR,1992-93,F,Loyola (IL),MW Coll,7,NaN,8,...,16,NaN,NaN,20,0,2,1,NaN,NaN,24
7,7,Gerrod Abram,JR,1992-93,G,Boston College,Big East,20,NaN,61,...,51,NaN,NaN,54,30,19,1,NaN,NaN,192
8,8,Dexter Abrams,SR,1992-93,C,Georgia Southern,Southern,27,NaN,139,...,65,NaN,NaN,166,22,35,11,NaN,NaN,328
9,9,Mohammed Acha,JR,1992-93,F,Coastal Carolina,Big South,32,NaN,182,...,159,NaN,NaN,265,69,69,51,NaN,NaN,478


#### NBA
***

In [98]:
#Note: Data has been collected for Per Game Stats

pd.read_csv("NBA_raw.csv")

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1990,Mark Acres,C,27,ORL,80,50,21.1,1.7,3.6,...,.692,1.9,3.5,5.4,0.8,0.5,0.3,0.9,3.1,4.5
1,1990,Michael Adams,PG,27,DEN,79,74,34.1,5.0,12.5,...,.850,0.6,2.2,2.8,6.3,1.5,0.0,1.8,1.7,15.5
2,1990,Mark Aguirre,SF,30,DET,78,40,25.7,5.6,11.5,...,.756,1.5,2.4,3.9,1.9,0.4,0.2,1.6,2.6,14.1
3,1990,Danny Ainge,PG,30,SAC,75,68,36.4,6.7,15.4,...,.831,0.9,3.4,4.3,6.0,1.5,0.2,2.5,3.2,17.9
4,1990,Mark Alarie,PF,26,WSB,82,10,23.1,4.5,9.6,...,.812,1.8,2.7,4.6,1.7,0.7,0.5,1.2,2.7,10.5
5,1990,Steve Alford,PG,25,DAL,41,0,7.4,1.5,3.4,...,.946,0.0,0.6,0.6,1.0,0.4,0.1,0.4,0.5,4.1
6,1990,Randy Allen,SG,25,SAC,63,6,11.8,1.7,3.8,...,.535,0.8,1.4,2.2,0.4,0.3,0.3,0.4,1.6,3.7
7,1990,Greg Anderson,PF,25,MIL,60,28,21.5,3.7,7.2,...,.535,1.9,4.4,6.2,0.4,0.5,0.9,1.3,2.9,8.8
8,1990,Nick Anderson,SG,22,ORL,81,9,22.0,4.6,9.3,...,.705,1.3,2.6,3.9,1.5,0.9,0.4,1.7,1.7,11.5
9,1990,Richard Anderson,PF,29,CHH,54,2,11.2,1.6,3.9,...,.783,0.6,1.7,2.4,1.0,0.4,0.2,0.5,1.2,4.3
